In [15]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

In [1]:
!pip install wandb


In [16]:
import wandb
wandb.login()

True

In [18]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/cascade_rcnn/cascade_rcnn_swin_fpn_1x_coco.py')




root='../../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/cascade_rcnn_swin_fpn_1x_trash'

cfg.model.roi_head.bbox_head[0].num_classes = 10
cfg.model.roi_head.bbox_head[1].num_classes = 10
cfg.model.roi_head.bbox_head[2].num_classes = 10


cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [19]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [20]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [21]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2024-01-17 01:00:57,303 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth
Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth" to /data/ephemeral/home/.cache/torch/hub/checkpoints/swin_tiny_patch4_window7_224.pth
100%|██████████| 109M/109M [00:01<00:00, 87.3MB/s] 
2024-01-17 01:01:00,193 - mmdet - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-01-17 01:01:00,214 - mmdet - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-01-17 01:01:00,220 - mmdet - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override': [{'name': 'shared_fcs'}, {'name': 'cls

In [22]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2024-01-17 01:01:04,560 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-01-17 01:01:04,563 - mmdet - INFO - Start running, host: root@instance-5492, work_dir: /data/ephemeral/home/baseline/mmdetection/work_dirs/cascade_rcnn_swin_fpn_1x_trash
2024-01-17 01:01:04,564 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook  

2024-01-17 01:01:16,360 - mmdet - WARNING - No meta information found in the runner. 
2024-01-17 01:01:16,362 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2024-01-17 01:01:34,862 - mmdet - INFO - Epoch [1][50/1221]	lr: 1.978e-03, eta: 1:30:01, time: 0.370, data_time: 0.050, memory: 6652, loss_rpn_cls: 0.3706, loss_rpn_bbox: 0.0437, s0.loss_cls: 0.5776, s0.acc: 89.1465, s0.loss_bbox: 0.0969, s1.loss_cls: 0.2032, s1.acc: 96.3965, s1.loss_bbox: 0.0285, s2.loss_cls: 0.1147, s2.acc: 89.9756, s2.loss_bbox: 0.0040, loss: 1.4392, grad_norm: 9.0080
2024-01-17 01:01:52,108 - mmdet - INFO - Epoch [1][100/1221]	lr: 3.976e-03, eta: 1:26:40, time: 0.345, data_time: 0.007, memory: 6652, loss_rpn_cls: 0.1584, loss_rpn_bbox: 0.0420, s0.loss_cls: 0.2931, s0.acc: 94.3652, s0.loss_bbox: 0.1309, s1.loss_cls: 0.0795, s1.acc: 97.5215, s1.loss_bbox: 0.0372, s2.loss_cls: 0.0256,

learning_rate,▄▇█████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/grad_norm,██▇▅▅▃▅▄▅▃▄▂▄▂▂▃▂▄▄▂▄▂▄▂▃▃▃▁▁▂▂▁▂▂▂▂▁▂▂▁
train/loss,▅▇▆██▇▇▆█▅▇▆▇▅▅▅▅▇▅▄▄▄▆▄▄▃▅▃▃▃▂▂▃▂▂▂▂▂▂▁
train/loss_rpn_bbox,▇▇▇██▇▅▅▆▅▆▅▆▅▄▅▅▆▅▅▅▄▄▃▅▃▅▃▃▂▂▂▃▂▂▂▃▃▂▁
train/loss_rpn_cls,██▆▆▆▅▅▄▄▅▄▃▄▄▃▃▃▃▃▃▃▂▃▂▂▂▃▂▂▂▁▁▂▁▁▁▂▁▁▁
train/s0.acc,▅▄▄▂▁▂▂▃▁▃▂▃▁▄▄▃▄▁▄▄▅▄▂▄▅▆▄▅▆▆▇▇▆▆▇▇▆▆▇█
train/s0.loss_bbox,▇▇▆██▇▆▅█▅▆▅▇▄▄▅▄▆▄▃▃▄▆▃▄▂▄▃▃▃▂▂▃▃▂▂▃▃▂▁
train/s0.loss_cls,▇█▇███▇▆█▆▇▆▇▅▅▆▆▆▅▅▄▄▆▅▄▃▅▃▃▃▂▂▃▂▂▂▂▂▂▁
train/s1.acc,█▄▄▂▁▂▁▂▁▃▂▂▁▃▃▂▃▁▃▃▄▃▂▃▃▄▃▄▅▅▅▆▅▅▅▅▅▅▆▆
train/s1.loss_bbox,▁▅▅▇█▇▇▆█▅▇▆▇▅▅▆▆▇▅▅▅▅▆▅▅▄▅▅▄▅▄▄▅▄▄▄▄▅▄▃
